In [41]:
# So we can load files from other sub-directories, e.g. datasets.
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
from torch import nn
import utils.utils as utils
import json
import numpy as np
from scipy.special import softmax
from sklearn.linear_model import LogisticRegression

import calibration as cal


In [2]:
print(f'cuda is available: {torch.cuda.is_available()}')
torch.cuda.empty_cache()

cuda is available: True


In [8]:
USE_CUDA = True
cuda = torch.device('cuda') 
batch_size, num_workers = 32, 16

In [71]:
def load_model(config_path, checkpoint_path):
    with open(config_path) as f:
        config = json.load(f)
    net = utils.initialize(config['model'])
    if USE_CUDA:
        net = net.cuda()
    net.new_last_layer(config['num_classes'])
    utils.load_ckp(checkpoint_path, net)
    return net

# Load datasets.
def load_test_dataset(config, idx, split_arg_name='split', split='val'):
    test_config = config['test_datasets'][idx]
    test_config['args'][split_arg_name] = split
    print(test_config['name'], test_config)
    if 'transforms' not in test_config:
        test_config['transforms'] = config['default_test_transforms']
    test_data = utils.init_dataset(test_config)
    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=batch_size,
        shuffle=False, num_workers=num_workers)
    return test_data, test_loader

def get_features_labels(net, loader):
    net.cuda()
    net.eval()
    feature_model = nn.Sequential(*list(net._model.children())[:-1])
    features_list, labels_list = [], []
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            features = feature_model(images)
            features_list.append(features.detach().cpu().numpy())
            labels_list.append(labels.detach().cpu().numpy())
    features = np.squeeze(np.concatenate(features_list))
    labels = np.concatenate(labels_list)
    return features, labels

def get_acc(preds, labels):
    return np.mean(preds == labels)

def make_none_list(rs, cs):
    return [[None] * cs for _ in range(rs)]

In [109]:
# Given a network how to get representations
def get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices, split_arg_names, split_names):
    M, L = len(model_names), len(loader_names)
    models = []
    for m in range(M):
        models.append(load_model(config_paths[m], checkpoint_paths[m]))

    with open(config_paths[0]) as f:
        config = json.load(f)
    loaders = []
    for l in range(L):
        _, loader = load_test_dataset(config, loader_indices[l], split_arg_names[l], split_names[l])
        loaders.append(loader)
    features, labels = make_none_list(M, L), make_none_list(M, L)
    for m in range(M):
        for l in range(L):
            features[m][l], labels[m][l] = get_features_labels(models[m], loaders[l])    
    return features, labels
        
def get_test_accs(features, labels, C=1.0):
    M, L = len(features), len(features[0])
    assert(L == 2)  # Only support train on first, validate on second for now
    clfs = []
    preds = []
    accs = []
    for m in range(M):
        cur_clf = LogisticRegression(random_state=0, C=C).fit(features[m][0], labels[m][0])
        cur_preds = cur_clf.predict(features[m][1])
        cur_acc = get_acc(cur_preds, labels[m][1])
        print(cur_acc)
        clfs.append(cur_clf)
        preds.append(cur_preds)
        accs.append(cur_acc)
    return clfs, preds, accs


In [81]:
# Try linear and fine-tuning models which have very different ID and OOD accs.
# On CIFAR, Im-n-C10

config_paths = [
    '../logs/breeds_moco_ft_0.003/config.json',
    '../logs/breeds_moco_ft_0.003/config.json'
]

checkpoint_paths = [
    '../logs/breeds_moco_ft_0.003/checkpoints/ckp_0',
    '../logs/breeds_moco_ft_0.003/checkpoints/ckp_last'
]

model_names = ['original_model', 'ft_model']
loader_names = ['l17_trg_train', 'l17_trg_val']
split_names = ['train', 'val']
split_arg_names = ['split', 'split']
loader_indices = [1, 1]  # Relative to the config file in the first config path in config_paths.

features, labels = get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
    split_arg_names, split_names)

target_val_living {'name': 'target_val_living', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'train', 'breeds_name': 'living17', 'root': '/u/scr/nlp/imagenet/'}}
target_val_living {'name': 'target_val_living', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'val', 'breeds_name': 'living17', 'root': '/u/scr/nlp/imagenet/'}, 'transforms': [{'classname': 'torchvision.transforms.Resize', 'args': {'size': [224, 224]}}, {'classname': 'torchvision.transforms.ToTensor'}, {'classname': 'torchvision.transforms.Normalize', 'args': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}}]}


In [111]:
clfs, preds, accs = get_test_accs(features, labels)

(44200, 2048)


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(44200, 2048)


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


([LogisticRegression(random_state=0), LogisticRegression(random_state=0)],
 [array([ 0,  0,  0, ..., 16, 16, 16]), array([ 0,  0,  0, ..., 16, 16, 16])],
 [0.9570588235294117, 0.8823529411764706])

In [ ]:
clfs, preds, accs = get_test_accs(features, labels, C=1.0)
print(accs)

In [58]:
net = load_model(config_paths[0], checkpoint_paths[0])
with open(config_paths[0]) as f:
        config = json.load(f)
_, loader = load_test_dataset(config, loader_indices[0], split_arg_names[1], split_names[1])
features, labels = get_features_labels(net, loader)
features = np.squeeze(features)

target_val_living


In [75]:
clf = LogisticRegression(random_state=0).fit(features, labels)
preds = clf.predict(features)

In [78]:
get_acc(preds, labels)

0.96

In [59]:
features.shape
# clf = LogisticRegression(random_state=0).fit(features, labels)

(1700, 2048)

In [36]:
feature_model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
